In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
 

from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
import csv
import os
import shutil

In [ ]:
csv_file_path = '/kaggle/input/ai-1803-dpl-302-m-butterfly-image-classification/Training_set.csv'
train_images_dir = '/kaggle/input/ai-1803-dpl-302-m-butterfly-image-classification/train/train'
test_images_dir = '/kaggle/input/ai-1803-dpl-302-m-butterfly-image-classification/test/test'

In [ ]:
with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        image_name = row[0]
        class_name = row[1]

        # Tạo thư mục cho lớp nếu chưa tồn tại
        class_dir = os.path.join('/kaggle/working', class_name)
        os.makedirs(class_dir, exist_ok=True)

        # Di chuyển hình ảnh vào thư mục của lớp tương ứng
        source = os.path.join(train_images_dir, image_name)
        destination = os.path.join(class_dir, image_name)
        
        try:
            shutil.copy(source, destination)  # Sử dụng copy thay vì move để tránh mất dữ liệu gốc
        except Exception as e:
            print(f"Error moving file {source} to {destination}: {e}")

In [ ]:
train2_dir = '/kaggle/working/Train2'
os.makedirs(train2_dir, exist_ok=True)
with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        class_name = row[1]
        source = os.path.join('/kaggle/working', class_name)
        try:
            shutil.move(source, train2_dir)
        except Exception as e:
            print(f"Error moving directory {source} to {train2_dir}: {e}")

In [ ]:
import pathlib
data_dir = pathlib.Path('/kaggle/working/Train2/')

In [ ]:
data_dir = pathlib.Path(train2_dir)
for dirname, _, filenames in os.walk(data_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))
def delete_folder(folder_path):
    try:
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    except OSError as e:
        print(f"Error: {e}")

# Ví dụ xóa thư mục cụ thể
folder_path_to_delete = os.path.join(train2_dir, "label")
delete_folder(folder_path_to_delete)

In [ ]:
img_height,img_width=224,224
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
VGG_model = Sequential()

pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(224,224,3),
                   pooling='max',classes=75,
                   weights='imagenet')


VGG_model.add(pretrained_model)
VGG_model.add(Flatten())
VGG_model.add(Dense(512, activation='relu'))
VGG_model.add(BatchNormalization())  # Batch Normalization layer
VGG_model.add(Dropout(0.5))

VGG_model.add(Dense(75, activation='softmax'))
pretrained_model.trainable=False

In [ ]:
VGG_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=35
history = VGG_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
import os

test_images_dir = '/kaggle/input/ai-1803-dpl-302-m-butterfly-image-classification/test/test'
allowed_formats = ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

test_image_paths = []
for root, dirs, files in os.walk(test_images_dir):
    for file in files:
        if file.lower().endswith(allowed_formats):
            test_image_paths.append(os.path.join(root, file))

print(f"Found {len(test_image_paths)} images in the test directory.")
for path in test_image_paths[:10]:  # In ra 10 đường dẫn đầu tiên
    print(path)


In [ ]:
from tensorflow.keras.preprocessing import image

# Tạo danh sách tên file
test_filenames = [os.path.basename(path) for path in test_image_paths]

# Dự đoán nhãn cho từng hình ảnh
predicted_labels = []
for img_path in test_image_paths:
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Tạo batch có kích thước 1

    predictions = VGG_model.predict(img_array)
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]
    predicted_labels.append(class_names[predicted_class])

# Tạo dataframe từ kết quả dự đoán
results_df = pd.DataFrame({
    'ID': test_filenames,
    'label': predicted_labels
})

# Lưu kết quả ra file CSV
results_df.to_csv('/kaggle/working/predictions_label.csv', index=False)

print("Predictions saved to predictions.csv")
